In [5]:
import sys
from IPython.display import HTML, clear_output
from base64 import b64encode

sys.path.append('/home/ec2-user/SageMaker/')

import os
from fsgan.inference.swap import FaceSwapping
from fsgan.criterions.vgg_loss import VGGLoss

#@markdown This step should only be done once unless one of the
#@markdown following parameters is changed:

#@markdown ---
#@markdown Path to the weights directory (make sure it is correct):
weights_dir = '/home/ec2-user/SageMaker/fsgan/weights/' #@param {type:"string"}
#@markdown Number of finetune iterations on the source subject:
finetune_iterations = 800 #@param {type:"slider", min:100, max:2000, step:1}
#@markdown If True, the inner part of the mouth will be removed from the segmentation:
seg_remove_mouth = True #@param {type:"boolean"}
#@markdown ---


detection_model = os.path.join(weights_dir, 'v2/WIDERFace_DSFD_RES152.pth')
pose_model = os.path.join(weights_dir, 'shared/hopenet_robust_alpha1.pth')
lms_model = os.path.join(weights_dir, 'v2/hr18_wflw_landmarks.pth')
seg_model = os.path.join(weights_dir, 'v2/celeba_unet_256_1_2_segmentation_v2.pth')
reenactment_model = os.path.join(weights_dir, 'v2/nfv_msrunet_256_1_2_reenactment_v2.1.pth')
completion_model = os.path.join(weights_dir, 'v2/ijbc_msrunet_256_1_2_inpainting_v2.pth')
blending_model = os.path.join(weights_dir, 'v2/ijbc_msrunet_256_1_2_blending_v2.pth')
criterion_id_path = os.path.join(weights_dir, 'v2/vggface2_vgg19_256_1_2_id.pth')
criterion_id = VGGLoss(criterion_id_path)

In [6]:
import os
import argparse
import sys
import pickle
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm
import numpy as np
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import RandomSampler, DataLoader
from fsgan.preprocess.preprocess_video import VideoProcessBase, base_parser
from fsgan.utils.obj_factory import obj_factory
from fsgan.utils.utils import load_model
from fsgan.utils.img_utils import bgr2tensor, tensor2bgr, create_pyramid
from fsgan.utils.landmarks_utils import LandmarksHeatMapDecoder
from fsgan.utils.seg_utils import blend_seg_label, SoftErosion
from fsgan.datasets.img_lms_pose_transforms import RandomHorizontalFlip, Rotate, Pyramids, ToTensor, Normalize
from fsgan.datasets import img_lms_pose_transforms
from fsgan.datasets.seq_dataset import SeqInferenceDataset, SingleSeqRandomPairDataset
from fsgan.datasets.appearance_map import AppearanceMapDataset
from fsgan.utils.video_renderer import VideoRenderer
from fsgan.utils.batch import main as batch
from fsgan.preprocess.crop_image_sequences import main as crop_image_sequences_main
from fsgan.preprocess.crop_video_sequences import main as crop_video_sequences_main
from fsgan.preprocess.detections2sequences_center import main as detections2sequences_main

import os
import math
import pickle
from tqdm import tqdm
import time
import numpy as np
import cv2
import glob
import torch
import torch.nn as nn
import pathlib
# from face_ssd import build_ssd
from face_detection_dsfd.face_ssd_infer import SSD
from face_detection_dsfd.data import widerface_640, TestBaseTransform
from face_detection_dsfd.layers.functions.detection import Detect
import time

In [7]:
def test_base_transform(image, mean):
    #x = cv2.resize(image, (size, size)).astype(np.float32)
    x = image.astype(np.float32)
    x -= mean
    x = x.astype(np.float32)
    return x


class TestBaseTransform:
    def __init__(self, mean):
        #self.size = size
        self.mean = np.array(mean, dtype=np.float32)

    def __call__(self, image, boxes=None, labels=None):
        return test_base_transform(image, self.mean), boxes, labels



class imageDataset(torch.utils.data.Dataset):
    def __init__(self, dataset_path, transform=TestBaseTransform((104, 117, 123))):
        self.transform = transform
        self.dataset_path = dataset_path
        self.image_list = glob.glob(os.path.join(dataset_path,'*'))
        self.data_num = len(self.image_list)

    def __len__(self):
        return self.data_num

    def __getitem__(self, idx):
        path = self.image_list[idx]
        img = cv2.imread(path)
        img_tensor = torch.from_numpy(self.transform(img)[0]).permute(2, 0, 1)
        
        return img_tensor, img, path

In [8]:
class imagesDetector(object):
    def __init__(self,data_dir , out_postfix='_dsfd.pkl', detection_model_path='weights/WIDERFace_DSFD_RES152.pth',
                 gpus=None, batch_size=1):
#         super(imageDetector, self).__init__()
        # assert torch.cuda.is_available(), 'CUDA must be available!'
        self.out_postfix = out_postfix
        self.batch_size = batch_size
        
        # initialize datsaset
        self.dataset = imageDataset(dataset_path)
        self.dataloader = torch.utils.data.DataLoader(self.dataset, batch_size=batch_size, shuffle=False)

        # Set default tensor type
        # torch.set_default_tensor_type('torch.cuda.FloatTensor')
        
        # Initialize configuration
        self.cfg = widerface_640
        self.thresh = self.cfg['conf_thresh']
        
        # Initialize device
        torch.set_grad_enabled(False)
        self.device, self.gpus = set_device(gpus, silence=True)

        # Initialize detection model
        self.net = SSD("test").to(self.device)
        self.net.load_state_dict(torch.load(detection_model_path))
        self.net.eval()
        
        # Support multiple GPUs
        if self.gpus and len(self.gpus) > 1:
            self.net = nn.DataParallel(self.net, self.gpus)
            
        self.shrink = 1.0

        # Reset default tensor type
        # torch.set_default_tensor_type('torch.FloatTensor')

    def __call__(self, input_path, output_dir,is_display=True):
        # 出力パスの確認
        print('output output_dir', output_dir)
        
        # デフォルトのテンソルタイプの指定
        torch.set_default_tensor_type('torch.cuda.FloatTensor')
        
        # ディテクションとセーブ
        for image_tensors, images, pathes in tqdm(self.dataloader):
            image_tensors = image_tensors.to(self.device)
            
            # detection 
            st = time.time()
            detections_batch = self.net(image_tensors)
            print('detection time',time.time()-st)

            for detections, img, path in zip(detections_batch,images,pathes):              
                img = img.cpu().numpy()
                detections = detections.unsqueeze(0)
                image_size = (img.shape[0],img.shape[1])
                output_path= os.path.join(output_dir, os.path.splitext(os.path.basename(path))[0])+'_bbox.pkl'
                bbox_img_path= os.path.join(output_dir, os.path.splitext(os.path.basename(path))[0])+'_bbox.jpg'
                
                det = []
                shrink = self.shrink 
                scale = torch.Tensor([image_size[1] / shrink, image_size[0] / shrink,
                                      image_size[1] / shrink, image_size[0] / shrink])
                
                # スケール変換？
                for i in range(detections.size(1)):
                    j = 0
                    while detections[0, i, j, 0] >= self.thresh:
                        curr_det = detections[0, i, j, [1, 2, 3, 4, 0]].cpu().numpy()
                        curr_det[:4] *= scale.cpu().numpy()

                        det.append(curr_det)
                        j += 1

                # もし検出されれば
                if len(det) == 0:
                    det_list.append(np.array([], dtype='float32'))
                else:
                    det = np.row_stack((det))
                    det_filtered = np.array(det[det[:, 4] > 0.5, :4])
    

                # 表示
                if is_display == True:
                    img = np.ascontiguousarray(img)
                    det_display = np.round(det_filtered).astype(int)
                    for rect in det_display:
                        cv2.rectangle(img, tuple(rect[:2]), tuple(rect[2:]), (0, 0, 255), 1)
                    cv2.imwrite(bbox_img_path, img)
                    if cv2.waitKey(1) & 0xFF == ord('q'):
                        break
                    
                # Write to file
                print('output_path',output_path)
                with open(output_path, 'wb') as f:
                    pickle.dump(det_filtered.flatten(), f)
                    
        torch.set_default_tensor_type('torch.FloatTensor')   
            
        
def set_device(gpus=None, use_cuda=True, silence=False):
    use_cuda = torch.cuda.is_available() if use_cuda else use_cuda
    if use_cuda:
        gpus = list(range(torch.cuda.device_count())) if not gpus else gpus
        if not silence:
            print('=> using GPU devices: {}'.format(', '.join(map(str, gpus))))
    else:
        gpus = None
        if not silence:
            print('=> using CPU device')
    device = torch.device('cuda:{}'.format(gpus[0])) if gpus else torch.device('cpu')

    return device, gpus

In [9]:
parser = argparse.ArgumentParser(description=__doc__, formatter_class=argparse.ArgumentDefaultsHelpFormatter,
                                 parents=[base_parser])
parser.add_argument('source', metavar='SOURCE', nargs='+',
                    help='image or video per source: files, directories, file lists or queries')
parser.add_argument('-t', '--target', metavar='TARGET', nargs='+',
                    help='video per target: files, directories, file lists or queries')
parser.add_argument('-o', '--output', metavar='DIR',
                    help='output directory')
parser.add_argument('-ss', '--select_source', default='longest', metavar='STR',
                    help='source selection method ["longest" | sequence number]')
parser.add_argument('-st', '--select_target', default='longest', metavar='STR',
                    help='target selection method ["longest" | sequence number]')
parser.add_argument('-b', '--batch_size', default=8, type=int, metavar='N',
                    help='mini-batch size')
parser.add_argument('-rm', '--reenactment_model', metavar='PATH',
                    default='../weights/nfv_msrunet_256_1_2_reenactment_v2.1.pth', help='reenactment model')
parser.add_argument('-cm', '--completion_model', default='../weights/ijbc_msrunet_256_1_2_inpainting_v2.pth',
                    metavar='PATH', help='completion model')
parser.add_argument('-bm', '--blending_model', default='../weights/ijbc_msrunet_256_1_2_blending_v2.pth',
                    metavar='PATH', help='blending model')
parser.add_argument('-ci', '--criterion_id', default="vgg_loss.VGGLoss('../weights/vggface2_vgg19_256_1_2_id.pth')",
                    metavar='OBJ', help='id criterion object')
parser.add_argument('-mr', '--min_radius', default=2.0, type=float, metavar='F',
                    help='minimum distance between points in the appearance map')
parser.add_argument('-oc', '--output_crop', action='store_true',
                    help='output crop around the face instead of full frame')
parser.add_argument('-rp', '--renderer_process', action='store_true',
                    help='If True, the renderer will be run in a separate process')

finetune = parser.add_argument_group('finetune')
finetune.add_argument('-f', '--finetune', action='store_true',
                      help='Toggle whether to finetune the reenactment generator (default: False)')
finetune.add_argument('-fi', '--finetune_iterations', default=800, type=int, metavar='N',
                      help='number of finetune iterations')
finetune.add_argument('-fl', '--finetune_lr', default=1e-4, type=float, metavar='F',
                      help='finetune learning rate')
finetune.add_argument('-fb', '--finetune_batch_size', default=4, type=int, metavar='N',
                      help='finetune batch size')
finetune.add_argument('-fw', '--finetune_workers', default=4, type=int, metavar='N',
                      help='finetune workers')
finetune.add_argument('-fs', '--finetune_save', action='store_true',
                      help='enable saving finetune checkpoint')
d = parser.get_default

In [10]:
class FaceData(object):
    def __init__(self, resolution=d('resolution'), crop_scale=d('crop_scale'), gpus=d('gpus'),
        cpu_only=d('cpu_only'), display=d('display'), verbose=d('verbose'), encoder_codec=d('encoder_codec'),
        # Detection arguments:
        detection_model=d('detection_model'), det_batch_size=d('det_batch_size'), det_postfix=d('det_postfix'),
        # Sequence arguments:
        iou_thresh=d('iou_thresh'), min_length=d('min_length'), min_size=d('min_size'),
        center_kernel=d('center_kernel'), size_kernel=d('size_kernel'), smooth_det=d('smooth_det'),
        seq_postfix=d('seq_postfix'), write_empty=d('write_empty'),
        # Pose arguments:
        pose_model=d('pose_model'), pose_batch_size=d('pose_batch_size'), pose_postfix=d('pose_postfix'),
        cache_pose=d('cache_pose'), cache_frontal=d('cache_frontal'), smooth_poses=d('smooth_poses'),
        # Landmarks arguments:
        lms_model=d('lms_model'), lms_batch_size=d('lms_batch_size'), landmarks_postfix=d('landmarks_postfix'),
        cache_landmarks=d('cache_landmarks'), smooth_landmarks=d('smooth_landmarks'),
        # Segmentation arguments:
        seg_model=d('seg_model'), smooth_segmentation=d('smooth_segmentation'),
        segmentation_postfix=d('segmentation_postfix'), cache_segmentation=d('cache_segmentation'),
        seg_batch_size=d('seg_batch_size'), seg_remove_mouth=d('seg_remove_mouth'),
        # Swapping arguments:
        batch_size=d('batch_size'), reenactment_model=d('reenactment_model'), completion_model=d('completion_model'),
        blending_model=d('blending_model'), criterion_id=d('criterion_id'), min_radius=d('min_radius'),
        output_crop=d('output_crop'), renderer_process=d('renderer_process'),
        # BBOX
        dataset_path='/',bbox_batch_size=1):
        self.batch_size = batch_size
        self.min_radius = min_radius
        self.output_crop = output_crop
        
        # General
        self.resolution = resolution
        self.crop_scale = crop_scale
        self.display = display
        self.verbose = verbose
        self.batch_size = batch_size

        # Detection
        self.dataset_path = dataset_path
        self.image_detector = imagesDetector(dataset_path, det_postfix, detection_model, gpus, bbox_batch_size)
        self.det_postfix = det_postfix
        
        # Initialize device
        torch.set_grad_enabled(False)
        self.device, self.gpus = set_device(gpus, not cpu_only)

    def __del__(self):
        if hasattr(self, 'video_renderer'):
            self.video_renderer.kill()

    def detect_faces(self, output_dir=None,select_source='longest'):
        input_path = self.dataset_path
        if output_dir is None:
            output_dir = os.path.splitext(input_path)[0]
            output_dir = output_dir[:-1]+'_bbox' if output_dir[-1] == '/' else output_dir+'_bbox'
       
        # Create directory
        if not os.path.isdir(output_dir):
            os.mkdir(output_dir)

        # Face detection
        self.image_detector(input_path, output_dir)
        
        

In [14]:
dataset_path = '/home/ec2-user/SageMaker/datasets/frontal_faces/frontal01/'
face_swapping = FaceData(
    detection_model=detection_model, pose_model=pose_model, lms_model=lms_model,
    seg_model=seg_model, reenactment_model=reenactment_model,
    completion_model=completion_model, blending_model=blending_model,
    criterion_id=criterion_id,
    seg_remove_mouth=finetune_iterations, batch_size=16, seg_batch_size=48,
    encoder_codec='mp4v',dataset_path=dataset_path,bbox_batch_size=1)

=> using GPU devices: 0


In [ ]:
face_swapping.detect_faces()


  0%|          | 0/344 [00:00<?, ?it/s]

output output_dir /home/ec2-user/SageMaker/datasets/frontal_faces/frontal01_bbox



  0%|          | 1/344 [00:06<36:19,  6.36s/it]

detection time 6.165854454040527
output_path /home/ec2-user/SageMaker/datasets/frontal_faces/frontal01_bbox/000150_bbox.pkl



  1%|          | 2/344 [00:07<26:52,  4.72s/it]

...

detection time 0.9342889785766602
output_path /home/ec2-user/SageMaker/datasets/frontal_faces/frontal01_bbox/000100_bbox.pkl



 28%|██▊       | 96/344 [02:48<06:08,  1.49s/it]

detection time 1.3246023654937744
output_path /home/ec2-user/SageMaker/datasets/frontal_faces/frontal01_bbox/000102_bbox.pkl



 28%|██▊       | 97/344 [02:49<05:49,  1.42s/it]

detection time 1.2014317512512207
output_path /home/ec2-user/SageMaker/datasets/frontal_faces/frontal01_bbox/000189_bbox.pkl
